In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [3]:
!pip install presidio_analyzer presidio_anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.2 MB/s eta 0:00:00


In [79]:
import pandas as pd
from datasets import load_dataset
import re
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import AnalyzerEngine
import spacy
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json

In [5]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Data Cleaning and Preprocessing

In [6]:
# Define the function that loads all the datasets needed
def load_datasets():
  # Millennial Sentiment Interview Dataset from Kaggle
  # Answer
  df_msitd_answer = pd.read_excel('datasets/Interview_Database.xlsx', sheet_name="Data")
  # Question
  df_msitd_question = pd.read_excel('datasets/Interview_Database.xlsx', sheet_name="Question")

  # Interview Selection Dataset for sales position from Kaggle
  df_sales = pd.read_csv('datasets/Data - Base.csv')

  # Software Engineering Q&A Dataset from Hugging Face
  se_dataset = load_dataset("juasdexter/interview_questions")
  df_se = pd.DataFrame(se_dataset['train'])

  # Simulated Interview Dataset from Hugging Face
  # sim_dataset = load_dataset("SamRex39/AI-Interviewer", split='train')
  # df_sim = pd.DataFrame(sim_dataset)
  df_sim = pd.read_json('datasets/structured_interview_data.jsonl', lines=True)

  return {
      'm_answer': df_msitd_answer,
      'm_question': df_msitd_question,
      'sales': df_sales,
      'soft_eng': df_se,
      'simulated': df_sim
  }

In [7]:
# Load the datasets
datasets = load_datasets()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/224 [00:00<?, ?B/s]

interview_questions.csv:   0%|          | 0.00/257k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/602 [00:00<?, ? examples/s]

In [8]:
datasets['simulated']

,interviewer_name,interviewer_introduction,text,Answer,Question
0,Pihu,"Hello, I'm Pihu, and I'll be conducting your i...","[{'role': 'interviewer', 'content': ''}, {'rol...",NaN,NaN
1,NaN,NaN,"[{'role': 'interviewer', 'content': ''}, {'rol...","Sure, my name is [Sajal], and I have a backgro...",NaN
2,NaN,NaN,"[{'role': 'interviewer', 'content': 'That's gr...",NaN,"That's great to hear, [Sajal]. Let's dive into..."
3,NaN,NaN,"[{'role': 'interviewer', 'content': ''}, {'rol...",Certainly. I hold a Bachelor's degree in Data ...,NaN
4,NaN,NaN,"[{'role': 'interviewer', 'content': 'How do yo...",NaN,How do you stay updated with the latest develo...
...,...,...,...,...,...
67,NaN,NaN,"[{'role': 'interviewer', 'content': ''}, {'rol...",What I enjoy most about working in data scienc...,NaN
68,NaN,NaN,"[{'role': 'interviewer', 'content': 'Great, [S...",NaN,"Great, [Sajal]. We've covered a lot of ground ..."
69,NaN,NaN,"[{'role': 'interviewer', 'content': ''}, {'rol...","Thank you, Pihu, for the insightful interview....",NaN
70,NaN,NaN,"[{'role': 'interviewer', 'content': 'Absolutel...",NaN,"Absolutely, [Sajal]. Our team prides itself on..."


In [9]:
# Text Cleaning Process
def clean_text(text):
  # Standardize formats
  # Remove whitespace
  text = re.sub(r'\s+', ' ', text).strip().lower()
  # Remove special characters
  text = re.sub(r'[^\w\s.,;?!-]', '', text)

  # Remove filler words
  fillers = ['uh', 'um', 'ah', 'er', 'like', 'you know', 'well', 'i mean', 'basically', 'actually']
  pattern = r'\b(' + '|'.join(fillers) + r')\b'
  text = re.sub(pattern, '', text, flags=re.IGNORECASE)

  return text

In [10]:
# Initialize engine
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
# Anonymizer
def remove_pii(text):
  analyzer_results = analyzer.analyze(text=text, language='en')
  results = anonymizer.anonymize(text=text, analyzer_results=analyzer_results).text
  return results

In [12]:
# Normalized numerical scores schema
def normalize_scores(df, score_cols):
  for col in score_cols:
    # Using min and max values to normalize the scores to 1-5
    min_val = df[col].min()
    max_val = df[col].max()
    if max_val > min_val:
        df[col] = round(1 + 4 * (df[col] - min_val) / (max_val - min_val), 1)
  return df

In [13]:
# Process the daataset
def process_data(datasets):
  processed = {}

  # Processed Millennial Sentiment Interview Dataset Answer
  millennial = datasets['m_answer'].copy()
  text_cols = [f'MaQ{i}' for i in range(1,11)] + [f'MiQ{i}' for i in range(1,11)]
  for col in text_cols:
    if col in millennial.columns:
      millennial[col] = millennial[col].astype(str).fillna('').apply(clean_text).apply(remove_pii)
  processed['m_answer'] = millennial

  processed['m_question'] = datasets['m_question'].copy()
  processed['m_question']['Details'] = processed['m_question']['Details'].apply(clean_text)

  # Processed Sales Dataset
  sales = datasets['sales'].copy()
  # Take score columns
  df_sales_scores = datasets['sales'].select_dtypes(include='number')

  for col in df_sales_scores.columns:
    # Convert score columns to numeric
    sales[col] = pd.to_numeric(sales[col], errors='coerce')

  # Normalize scores
  sales = normalize_scores(sales, df_sales_scores.columns)

  # Take comments columns
  text_cols = ['Comments', 'RedFlags Comments in Interview']

  # Process text columns
  for col in text_cols:
    if col in sales.columns:
      sales[col] = sales[col].astype(str).fillna('').apply(clean_text)

  processed['sales'] = sales

  # Processed Software Engineering Interview Dataset
  se = datasets['soft_eng'].copy()

  # Process text columns
  se['question'] = se['question'].astype(str).fillna('').apply(clean_text).apply(remove_pii)
  se['answer'] = se['answer'].astype(str).fillna('').apply(clean_text).apply(remove_pii)

  processed['soft_eng'] = se

  # Processed Simulated Dataset
  sim = datasets['simulated'].copy()
  # Process text columns
  sim['Answer'] = sim['Answer'].astype(str).fillna('').apply(clean_text).apply(remove_pii)
  sim['Question'] = sim['Question'].astype(str).fillna('').apply(clean_text).apply(remove_pii)
  processed['simulated'] = sim

  return processed

In [24]:
processed_data = process_data(datasets)

In [81]:
# Save the cleaned datasets
for name, df in processed_data.items():
  df.to_csv(f'outputs/{name}_cleaned.csv', index=False)

### Behavioral Question Framework & Analysis

In [26]:
# Competency mapping
COMPETENCY_MAP = {
    'communication': ['express', 'explain', 'present', 'discuss', 'communicate'],
    'teamwork': ['collaborate', 'team', 'work together', 'support', 'help', 'cooperate', 'conflict', 'partner', 'member'],
    'problem_solving': ['solve', 'resolve', 'fix', 'troubleshoot', 'issue', 'problem', 'challenge'],
    'leadership': ['lead', 'manage', 'mentor', 'guide', 'direct', 'organize']
}

In [27]:
# STAR keywords for further detection
STAR_KEYWORDS = {
    'situation': ['when', 'situation', 'context', 'scenario', 'environment', 'during'],
    'task': ['task', 'responsibility', 'role', 'objective', 'goal', 'asked to', 'responsible for'],
    'action': ['action', 'approach', 'method', 'implement', 'utilized', 'decided', 'developed', 'explored', 'collected', 'construct', 'evaluated', 'conducted', 'identified'],
    'result': ['result', 'outcome', 'impact', 'achievement', 'conclusion', 'success']
}

In [28]:
# Map the text with correpsonding competency
def map_competency(text):
  competencies = []
  tokens = word_tokenize(text.lower())
  for comp, keywords in COMPETENCY_MAP.items():
    if any(keyword in tokens for keyword in keywords):
      competencies.append(comp)
  return competencies

In [29]:
# Detect if STAR keywords appears in the text
def detect_star_components(text):
  components = {key: False for key in STAR_KEYWORDS}
  tokens = word_tokenize(text.lower())
  for comp, keywords in STAR_KEYWORDS.items():
    if any(keyword in tokens for keyword in keywords):
      components[comp] = True
  return components

In [84]:
# Define STAR rubric
def create_star_rubric():
  text = {
      'Situation': {
          '1': 'No context or task background provided',
          '2': 'Partial context or task background provided but still missing details',
          '3': 'Clear and complete context or task background provided'
      },
      'Task': {
          '1': 'No specific task or responsibility provided',
          '2': 'Partial task or responsibility description provided but still missing details',
          '3': 'Clear and complete task or responsibility provided'
      },
      'Action': {
          '1': 'No specific task actions provided',
          '2': 'Partial task actions provided but still missing details',
          '3': 'Clear and complete task actions and work process provided'
      },
      'Result': {
          '1': 'No measurable outcomes provided',
          '2': 'Partial outcomes provided but still missing details',
          '3': 'Clear and complete outcomes and skills leanred provided'
      }
  }
  return text

In [85]:
# Define Competency rubric creation
def create_competency_rubric():
  rubrics = {}
  for competency in COMPETENCY_MAP.keys():
    rubrics[competency] ={
        '1': 'No evidence of competency demonstrated',
        '2': 'Demonstration of competency is below expectations and weak',
        '3': 'Demonstration of competency is meetting expectations and good',
        '4': 'Demonstration of competency is above expectations and strong',
        '5': 'Demonstration of competency is outstanding and exceptional'
    }
  return rubrics

In [72]:
# Get the question_bank from datasets
def create_question_bank(datasets):
  question_bank = []

  # Extract answers and questions from Millennial dataset
  millennial_answer = pd.read_csv('datasets/m_answer_cleaned.csv')
  millennial_question = pd.read_csv('datasets/m_question_cleaned.csv')
  questions_dict = dict(zip(millennial_question['Question'], millennial_question['Details']))

  # Get the competency and STAR components for each asnwer in the dataset
  for col in millennial_answer.columns:
    if col.startswith('MaQ') or col.startswith('MiQ'):
      for _, row in millennial_answer.iterrows():
        # Get the corresponding question and answer
        question = questions_dict.get(col, col)
        answer = row[col]
        if isinstance(answer, str):
          # Get the competency and STAR components
          competencies = map_competency(answer)
          star_components = detect_star_components(answer)
          question_bank.append({
              'source': 'Millennial Sentiment Interview Dataset',
              'question': clean_text(question),
              'answer': clean_text(answer),
              'competencies': competencies,
              'star': star_components
          })

  # Extract from Sales Dataset
  sales = pd.read_csv('datasets/sales_cleaned.csv')

  # Get the competency and STAR components for each asnwer in the dataset
  for _, row in sales.iterrows():
    question = "Sales scenarios"

    answer = (str(row['Call-pitch Elements used during the call Sales Scenario'])
    + str(row['But, my child\'s exam are going on now, so we will keep the counselling session after the exams get over.(Time: Favourable pitch: Counsellor hype)'])
    + str(row['Let me discuss it with my child'])
    + str(row['Sir being in education industry I know this is a marketing gimmick and at the end of the day you\'ll be selling the app.'])
    + str(row['Comments']) + " " + str(row['RedFlags Comments in Interview']))

    if isinstance(answer, str):
      competencies = map_competency(answer)
      star_components = detect_star_components(answer)
      question_bank.append({
          'source': 'Sales Dataset',
          'question': clean_text(question),
          'answer': clean_text(answer),
          'confidence_score': row.get('Confidence Score', 3),
          'structured_score': row.get('Structured Thinking Score', 3),
          'regional_fluency_score': row.get('Regional Fluency Score', 3),
          'total_score': row.get('Total Score', 3),
          'competencies': competencies,
          'star': star_components
      })

  # Extract from Software Engineering Interview Dataset
  se = pd.read_csv('datasets/soft_eng_cleaned.csv')
  # Get the competency and STAR components for each asnwer in the dataset
  for _, row in se.iterrows():
    competencies = map_competency(row['question'])
    star_components = detect_star_components(row['answer'])
    question_bank.append({
        'source': 'Software Engineering Interview Dataset',
        'question': row['question'],
        'answer': row['answer'],
        'competencies': competencies,
        'star': star_components
    })

  # Extract from Simulated Dataset
  sim = pd.read_csv('datasets/simulated_cleaned.csv')
  # Extract questions from Simulated Dataset
  df1 = pd.DataFrame({'Question': ['Tell me about Yourself']})
  df2 = sim['Question'].dropna().reset_index(drop=True)
  sim_question = pd.concat([df1, df2]).reset_index(drop=True)
  # Extract answers from Simulated Dataset
  sim_answer = sim['Answer'].dropna().reset_index(drop=True)

  sim_combined = pd.concat([sim_question, sim_answer], axis=1)
  # Get the competency and STAR components for each asnwer in the dataset
  for _, row in sim_combined.iterrows():
    competencies = map_competency(row['Question'])
    star_components = detect_star_components(row['Answer'])
    question_bank.append({
        'source': 'Simulated Dataset',
        'question': row['Question'],
        'answer': row['Answer'],
        'competencies': competencies,
        'star': star_components
    })

  return pd.DataFrame(question_bank)

In [73]:
question_bank = create_question_bank(datasets)

In [76]:
# Question categorization using clustering
def categorize_questions(question_bank):
  vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
  X = vectorizer.fit_transform(question_bank['question'])

  # Cluster into 8 categories
  kmeans = KMeans(n_clusters=4, random_state=42)
  question_bank['category'] = kmeans.fit_predict(X)

  # Map clusters to human-readable names
  cluster_names = {
      0: 'Communication',
      1: 'Teamwork',
      2: 'Problem Solving',
      3: 'Leadership'
  }
  question_bank['category'] = question_bank['category'].map(cluster_names)

  return question_bank

In [77]:
question_bank = categorize_questions(question_bank)

In [82]:
# Save the question bank
question_bank.to_csv('outputs/behavioral_question_bank.csv', index=False)

In [87]:
# Save competency and STAR rubrics
competency_rubrics = create_competency_rubric()
star_rubrics = create_star_rubric()

with open('outputs/competency_rubrics.json', 'w') as f:
  json.dump(competency_rubrics, f, indent=2)
with open('outputs/star_rubrics.json', 'w') as f:
  json.dump(star_rubrics, f, indent=2)